In [47]:
import requests
import tensorflow as tf
# import numpy as np
import pandas as pd
import json
import base64

In [103]:
data = pd.read_csv('data/heart_statlog_cleveland_hungary_final.csv')
first_data = data.sample()
test_data = first_data.iloc[:, :-1].to_dict()

# test_data = list(str(i) for i in test_data.values.tolist()[0])

print(first_data)


      age  sex  chest pain type  resting bp s  cholesterol  \
1093   58    1                4           128          259   

      fasting blood sugar  resting ecg  max heart rate  exercise angina  \
1093                    0            2             130                1   

      oldpeak  ST slope  target  
1093      3.0         2       1  


In [104]:
feature_spec = dict()
for keys, values in test_data.items():
    for keys_2, values_2 in values.items():
        # print(keys, values_2)
        if isinstance(values_2, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[int(values_2)]))
        elif isinstance(values_2, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[float(values_2)]))

example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

print(feature_spec)


{'age': int64_list {
  value: 58
}
, 'sex': int64_list {
  value: 1
}
, 'chest pain type': int64_list {
  value: 4
}
, 'resting bp s': int64_list {
  value: 128
}
, 'cholesterol': int64_list {
  value: 259
}
, 'fasting blood sugar': int64_list {
  value: 0
}
, 'resting ecg': int64_list {
  value: 2
}
, 'max heart rate': int64_list {
  value: 130
}
, 'exercise angina': int64_list {
  value: 1
}
, 'oldpeak': float_list {
  value: 3.0
}
, 'ST slope': int64_list {
  value: 2
}
}


In [120]:
result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

json_data = json.dumps(
    {
        "signature_name": "serving_default",
        "instances": result,
    }
)

endpoint = "http://103.190.215.239:8501/v1/models/cc-model:predict"
headers = {"content-type": "application/json"}

response = requests.post(endpoint, data=json_data, headers=headers)
prediction_result = json.loads(response.text)
kelas = ['Normal', 'Heart Desease']
print("Prediction : ", kelas[int(prediction_result['predictions'][0][0])])

Prediction :  Heart Desease
